In [24]:
#from datarobot import datarobot as dr

import datarobot as dr

dr.Client(endpoint='https://app.datarobot.com/api/v2', token='NjE1NTQzNTc4ZDVjNGEzMmFlYTg1YWIyOkZjK1Y5TnRRNmI5OGkwc1IvWHYwNC9OU24rUnJjbGVKYk5FLzhSelhvTzQ9')

dep_name='API Automation - 3'


In [25]:
custom_model=dr.CustomInferenceModel.create(
    name=dep_name,
    target_type=dr.TARGET_TYPE.MULTICLASS,
    target_name='class',
    class_labels=['Iris-setosa','Iris-versicolor','Iris-virginica'],
    description='Example of API automation',
    language='Python 3'
)
custom_model.id

'6177076ed60255664ac08c3b'

In [26]:
train_data='https://raw.githubusercontent.com/amperie/mlops/main/datasets/iris.csv'

dataset=dr.Dataset.create_from_url(url=train_data)
custom_model.assign_training_data(dataset.id)
custom_model.training_data_assignment_in_progress

False

In [27]:

envs=dr.ExecutionEnvironment.list()
ExecutionEnvironment=next((x for x in envs if x.name == '[DataRobot] Python 3 Scikit-Learn Drop-In'), None)
ExecutionEnvironment.id


'5e8c889607389fe0f466c72d'

In [28]:

model_version=dr.CustomModelVersion.create_clean(
    custom_model_id=custom_model.id,
    base_environment_id=ExecutionEnvironment.id,
    files=['svm.pkl'],
)


test_data = 'https://raw.githubusercontent.com/amperie/mlops/main/datasets/iris-notarget.csv'
dataset = dr.Dataset.create_from_url(url=test_data)


In [29]:
custom_model_test = dr.CustomModelTest.create(
    custom_model_id=custom_model.id,
    custom_model_version_id=model_version.id,
    dataset_id=dataset.id,
    max_wait=3600,  # 1 hour timeout
)

custom_model_test
custom_model_test.overall_status

'warning'

In [30]:

prediction_server = dr.PredictionServer.list()[0]
deployment=dr.Deployment.create_from_custom_model_version(
    model_version.id,
    dep_name,
    dep_name,
    prediction_server.id,
)

prediction_server

PredictionServer(https://datarobot-demo.orm.datarobot.com)